# Inference notebook

In [7]:
from data_load import AbalonDataset
from torch.utils.data import DataLoader

def load_data(batch_size):

    train_dataset, test_dataset = AbalonDataset.create_train_test_val("train.csv")
    val_dataset = AbalonDataset.create_train_test_val("test.csv", train_dataset)

    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    val_loader = DataLoader(test_dataset, batch_size=batch_size)
    test_loader = DataLoader(val_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader


### Run inference
run the code below to infer the predicted rings and add them as a new column

In [10]:
import torch
import torch.nn as nn
import pandas as pd

class AbaloneNetworkTwoLayer(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_inputs, num_hidden),
            nn.Tanh(),            
            nn.Linear(num_hidden, num_hidden),
            nn.Tanh(),
            nn.Linear(num_hidden, num_outputs))

    def forward(self, x):
        return self.model(x)

def create_predicted_csv(model_or_saved, test_loader, device):
    """if you set the model_or_saved to None or False, you use the torch pth file to load the model. You can also provide 
    model as parameter, then it will work based on the provided model"""
    if not model_or_saved:
        model = torch.load("best_model1.37.pth", weights_only=False)
    else:
        model = model_or_saved
    model.eval()
    final_predictions = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            final_predictions.extend([round(float(output), 2) for output in outputs])
    
    final_df = pd.read_csv("data/test.csv")
    final_df["prediction"] = final_predictions
    final_df.to_csv("data/final.csv")

    return "correctly saved into final_test.csv the predictions + original data"
device = torch.device("cpu")
trainloader, valloader, testloader = load_data(32)
print(create_predicted_csv(False, testloader, device))

correctly saved into final_test.csv the predictions + original data


/Users/mehdigreefhorst/Desktop/DeepLearning1stAssignment-abalone/data_load.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  numerical_tensor = torch.tensor(row, dtype=torch.float32)
